In [3]:

from pyomo.environ import *
import numpy as np
scenarios = range(1,4)
solver = SolverFactory('gurobi',tee=False)
model_mp = ConcreteModel()
model_mp.x = Var(scenarios,domain = NonNegativeReals)
model_mp.zw = Var(scenarios)
model_mp.obj = Objective(expr = 150*model_mp.x[1] +230*model_mp.x[2] +260*model_mp.x[3] + 1/3*sum(model_mp.zw[s] for s in scenarios), sense=minimize)
model_mp.con_extra = ConstraintList()
model_mp.constraint1 = Constraint(expr = model_mp.x[1]+model_mp.x[2]+model_mp.x[3] <= 500) 
yield_1 = {1:3,2:2.5,3:2}
yield_2 = {1:3.6,2:3,3:2.4}
yield_3 = {1:24,2:20,3:16}
model_mp.constraint_base = Constraint(scenarios, rule=lambda model_mp, s:model_mp.zw[s]>=-10000000)
solver.options['DualReductions'] = 0
#solver.solve(model_mp)
def subproblem_dual(x,s):
    model = ConcreteModel()
    model.dual = Suffix(direction=Suffix.IMPORT)
    model.y = Var(range(1,3),domain = NonNegativeReals)
    model.w = Var(range(1,5),domain = NonNegativeReals)
    model.s = Var(range(1,5),domain = NonNegativeReals)
    model.obj = Objective(expr = 238*model.y[1] - 170*model.w[1]
    +210*model.y[2] -150*model.w[2] -36*model.w[3] -10*model.w[4] , sense=minimize)
    model.constraint2 = Constraint(expr = yield_1[s]*x[0] +model.y[1] -model.w[1]-model.s[1] == 200)  
    model.constraint3 = Constraint(expr =  yield_2[s]*x[1] +model.y[2] -model.w[2] -model.s[2]== 240) 
    model.constraint4 = Constraint(expr = model.w[3] +model.w[4] +model.s[3] - yield_3[s]*x[2]==0)
    model.constraint5 = Constraint(expr = model.w[3] +model.s[4]==6000)
    dw = np.array([200,240,0,6000])
    Bw = np.array([[yield_1[s],0,0],[0,yield_2[s],0],[0,0,-yield_3[s]],[0,0,0]])
      # Use GLPK solver
    #solver.solve(model)
    #model.pprint()
    #a = np.array([model.dual[model.constraint2],model.dual[model.constraint3],model.dual[model.constraint4],model.dual[model.constraint5]])
    return dw,Bw,model
while(1):
    results = solver.solve(model_mp)
    print(results.solver.termination_condition)
    x = [model_mp.x[1](),model_mp.x[2](),model_mp.x[3]()]
    print(x)
    #model_mp.pprint()
    c = 0
    msp_obj_val = []
    for i in scenarios:
        dw,Bw,msp = subproblem_dual(x,i)
        solver.solve(msp)
        #print(i)
        #print(x)
        #print(msp.dual.display())
        a = np.array([msp.dual[msp.constraint2],msp.dual[msp.constraint3],msp.dual[msp.constraint4],msp.dual[msp.constraint5]])
        if(results.solver.termination_condition == TerminationCondition.infeasible):
            #print(a*dw)
            d = np.matmul(a,Bw)
            model_mp.con_extra.add(np.dot(a,dw)-sum(d[j-1]*model_mp.x[j] for j in range(1,4)) <= 0)
        else:
            if(msp.obj()<=model_mp.zw[i]()):
                c = c+1
                msp_obj_val.append(msp.obj())
            else:       
                d = np.matmul(a,Bw)
                model_mp.con_extra.add(np.dot(a,dw)-sum(d[j-1]*model_mp.x[j] for j in range(1,4)) <=model_mp.zw[i])
    if(c==3):
        print("yes")
        print(x)
        model_mp.pprint()
        print(model_mp.obj())
        break
            
    

optimal
[0.0, 0.0, 0.0]
optimal
[0.0, 0.0, 500.0]
optimal
[250.0, 0.0, 250.0]
optimal
[80.0, 174.35897435897436, 245.64102564102564]
optimal
[117.59172154280337, 115.50329256820318, 266.90498588899345]
optimal
[170.0, 80.0, 250.0]
yes
[170.0, 80.0, 250.0]
2 Var Declarations
    x : Size=3, Index={1, 2, 3}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 : 170.0 :  None : False :  True : NonNegativeReals
          2 :     0 :  80.0 :  None : False :  True : NonNegativeReals
          3 :     0 : 250.0 :  None : False :  True : NonNegativeReals
    zw : Size=3, Index={1, 2, 3}
        Key : Lower : Value     : Upper : Fixed : Stale : Domain
          1 :  None : -275900.0 :  None : False :  True :  Reals
          2 :  None : -218250.0 :  None : False :  True :  Reals
          3 :  None : -157720.0 :  None : False :  True :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None